In [1]:
import os
os.environ['VARIATION_NORM_EB_PROD'] = 'true'

from variation.mane_transcript import MANETranscript
from variation.tokenizers.caches import AminoAcidCache
from variation.data_sources import SeqRepoAccess, TranscriptMappings, MANETranscriptMappings, UTA
from pyliftover import LiftOver
import pandas as pd

In [2]:
transcript_mappings = TranscriptMappings()
amino_acid_cache = AminoAcidCache()
seqrepo = SeqRepoAccess()
mane_transcript_mappings = MANETranscriptMappings()
uta = UTA(db_pwd='admin')
mane_transcript = MANETranscript(seqrepo, transcript_mappings, mane_transcript_mappings, uta)

# Assembly mappings
GRCH_TO_HG = {
    'GRCh37': 'hg19',
    'GRCh38': 'hg38'
}

In [3]:
# Actual method name is get_tx_exon_aln_v_data, but simplified to make viewing easier
def get_tx_alt_range_data(ac, start_pos, end_pos):
    """Return queried data from tx_exon_aln_v table.

    :param str ac: Transcript accession
    :param int start_pos: Start genomic position change 
    :param int end_pos: End genomic position change
    :return: tx_exon_aln_v data
    """
    query = (
        f"""
        SELECT *
        FROM uta_20210129.tx_exon_aln_v
        WHERE tx_ac = '{ac}'
        AND alt_ac LIKE 'NC_00%'
        AND alt_aln_method = 'splign'
        AND {start_pos} BETWEEN alt_start_i AND alt_end_i
        AND {end_pos} BETWEEN alt_start_i AND alt_end_i
        ORDER BY alt_ac;
        """
    )
    return pd.read_sql(query, uta.conn)

# g -> MANE c

In [4]:
def g_to_mane_c(ac, start_pos, end_pos):
    """Return MANE Transcript on the c. coordinate.
    g->GRCh38->MANE c.

    :param str ac: Transcript accession on g. coordinate
    :param int start_pos: genomic change start position
    :param int end_pos: genomic change end position
    :return: MANE Transcripts with cDNA change on c. coordinate
    """
    if end_pos is None:
        end_pos = start_pos
    
    # Get gene symbol which is used to get MANE data (accessions)
    gene_symbol = uta.get_gene_from_ac(ac, start_pos, end_pos)
    if not gene_symbol:
        return None

    mane_data = mane_transcript_mappings.get_gene_mane_data(gene_symbol)
    if not mane_data:
        return None
    mane_data_len = len(mane_data)

    # Priority: MANE Select, MANE Clinical Plus 
    for i in range(mane_data_len):
        index = mane_data_len - i - 1
        current_mane_data = mane_data[index]

        mane_c_ac = current_mane_data['RefSeq_nuc']

        # Liftover
        grch38 = mane_transcript.g_to_grch38(ac, start_pos, end_pos)
        start_pos, end_pos = grch38['pos']

        # Get transcript and genomic data given genomic coordinates 
        mane_tx_genomic_data = uta.get_mane_c_genomic_data(
            mane_c_ac, None, start_pos, end_pos
        )
        if not mane_tx_genomic_data:
            return None

        # Find genomic shift and apply to transcript 
        tx_pos_range = mane_tx_genomic_data['tx_pos_range']
        alt_pos_change = mane_tx_genomic_data['alt_pos_change']
        coding_start_site = mane_tx_genomic_data['coding_start_site']

        mane_c_pos_change = (
            tx_pos_range[0] + alt_pos_change[0] - coding_start_site,
            tx_pos_range[1] - alt_pos_change[1] - coding_start_site
        )
    
        return {
            'alt_ac': ac,
            'alt_pos': (start_pos, end_pos),
            'tx_ac': mane_c_ac,
            'tx_pos': mane_c_pos_change
        }

## BRAF V600E example (walkthrough)

In [5]:
# NC_000007.13:g.140453136A>T
ac = 'NC_000007.13'
start_pos = 140453136
end_pos = 140453136

In [6]:
# Get gene from genomic accession
gene = uta.get_gene_from_ac(ac, start_pos, end_pos)    
gene

'BRAF'

In [7]:
# Use gene to get MANE data 
# (https://ftp.ncbi.nlm.nih.gov/refseq/MANE/MANE_human/current/MANE.GRCh38.v0.95.summary.txt.gz)
# Hiding implementation details since it's just finding row with symbol == 'BRAF'
# BRAF only has MANE Select data, so we can just select the first from the list
mane_data = mane_transcript_mappings.get_gene_mane_data(gene)[0]
mane_data

{'#NCBI_GeneID': 'GeneID:673',
 'Ensembl_Gene': 'ENSG00000157764.14',
 'HGNC_ID': 'HGNC:1097',
 'symbol': 'BRAF',
 'name': 'B-Raf proto-oncogene, serine/threonine kinase',
 'RefSeq_nuc': 'NM_001374258.1',
 'RefSeq_prot': 'NP_001361187.1',
 'Ensembl_nuc': 'ENST00000644969.2',
 'Ensembl_prot': 'ENSP00000496776.1',
 'MANE_status': 'MANE Select',
 'GRCh38_chr': '7',
 'chr_start': 140719337,
 'chr_end': 140924929,
 'chr_strand': '-'}

In [8]:
# Use RefSeq accession 
mane_c_ac = mane_data['RefSeq_nuc']
mane_c_ac

'NM_001374258.1'

In [9]:
# We get chromosome and assembly to see if we have to liftover to GRCh38
# uta.get_chr_assembly() will return None if it's already using GRCh38 
chromosome, assembly = uta.get_chr_assembly(ac)
chromosome, assembly

('chr7', 'GRCh37')

In [10]:
# We know we have to liftover
# Start and end are the same, so we only have to liftover start position 
lo = LiftOver(GRCH_TO_HG[assembly], 'hg38')
liftover = lo.convert_coordinate(chromosome, start_pos)[0][1]
start_pos, end_pos = liftover, liftover
start_pos, end_pos

(140753336, 140753336)

In [11]:
# Now we are going to get transcript range and nc accession range
data = get_tx_alt_range_data(mane_c_ac, start_pos, end_pos)
data

,hgnc,tx_ac,alt_ac,alt_aln_method,alt_strand,ord,tx_start_i,tx_end_i,alt_start_i,alt_end_i,cigar,tx_aseq,alt_aseq,tx_exon_set_id,alt_exon_set_id,tx_exon_id,alt_exon_id,exon_aln_id
0,BRAF,NM_001374258.1,NC_000007.14,splign,-1,15,2087,2206,140753274,140753393,119=,None,None,943125,1017563,8439617,9353476,5697247


In [12]:
# Get coding start site for tx_ac 
cds_start = uta.get_coding_start_site(data['tx_ac'].item())
cds_start

226

In [13]:
alt_pos_range = data['alt_start_i'].item(), data['alt_end_i'].item()
tx_pos_range = data['tx_start_i'].item(), data['tx_end_i'].item()
assert alt_pos_range[1] - alt_pos_range[0] == tx_pos_range[1] - tx_pos_range[0]

print(f"alt_pos_range: {alt_pos_range}")
print(f"tx_pos_range: {tx_pos_range}")

alt_pos_range: (140753274, 140753393)
tx_pos_range: (2087, 2206)


In [14]:
# Get difference for NC accession, so we can shift transcript 
alt_pos_diff = (start_pos-alt_pos_range[0], alt_pos_range[1]-end_pos)
alt_pos_diff

(62, 57)

In [15]:
# Shift transcript position using differences above
alt_pos_change = (tx_pos_range[0] + alt_pos_diff[0] - cds_start, tx_pos_range[1] - alt_pos_diff[1] - cds_start)
alt_pos_change

(1923, 1923)

In [16]:
# Using ClinGen Allele Registry API as a reference
# https://reg.genome.network/allele?hgvs=NC_000007.13:g.140453136A%3ET
# MANE transcript: NM_001374258.1:c.1919T>A
expected = (1919, 1919)
alt_pos_change == expected

False

## More examples

In [17]:
# BRAF V600E GRCh37
# Expect 1919, 1919
g_to_mane_c('NC_000007.13', 140453136, 140453136)

{'alt_ac': 'NC_000007.13',
 'alt_pos': (140753336, 140753336),
 'tx_ac': 'NM_001374258.1',
 'tx_pos': (1923, 1923)}

In [18]:
# BRAF V600E GRCh38
# Expect 1919, 1919
g_to_mane_c('NC_000007.14', 140753336, 140753336)

{'alt_ac': 'NC_000007.14',
 'alt_pos': (140753336, 140753336),
 'tx_ac': 'NM_001374258.1',
 'tx_pos': (1923, 1923)}

In [19]:
# EGFR L858R GRCh37
# https://reg.genome.network/allele?hgvs=NC_000007.13:g.55259515T%3EG
# Expect 2573, 2573
g_to_mane_c('NC_000007.13', 55259515, 55259515)

{'alt_ac': 'NC_000007.13',
 'alt_pos': (55191822, 55191822),
 'tx_ac': 'NM_005228.5',
 'tx_pos': (2573, 2573)}

In [20]:
# https://civicdb.org/events/genes/30/summary/variants/79/summary#variant
# https://reg.genome.network/allele?hgvs=NC_000012.11:g.25398284C%3ET
# Expect 35, 35
g_to_mane_c('NC_000012.11', 25398284, 25398284)

{'alt_ac': 'NC_000012.11',
 'alt_pos': (25245350, 25245350),
 'tx_ac': 'NM_004985.5',
 'tx_pos': (66, 66)}